In [3]:
import os
import sys

# Get the current working directory
current_dir = os.getcwd()
# Add the ./src folder to the Python module search path
sys.path.append(os.path.join(current_dir, '..', 'src'))

from utils import *
from optimize_symmetric import *

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

import xgboost as xgb

sns.set_style('ticks')

## 0. Read and inspect data files

In [4]:
input_path = '../Data'

feature_file = 'deepnose_features_UPD.npy'
CID_file = 'molecules_train_cid.npy'

mixture_file = 'Mixure_Definitions_Training_set.csv'
training_task_file = 'TrainingData_mixturedist.csv'

# Deepnose features
features = np.load(os.path.join(input_path, feature_file))
# Training dataframe
training_set = pd.read_csv(os.path.join(input_path, training_task_file))

# Mapping helper files
mixtures_IDs = pd.read_csv(os.path.join(input_path, mixture_file))
features_CIDs = np.load(os.path.join(input_path, CID_file))

#### Try out log standardization:

In [5]:
# Exponentiation
# features = np.exp(features)
# Standard transform features:
epsilon = 1e-8
scaler = StandardScaler(with_mean=True, with_std=True)
features = scaler.fit_transform(np.log(features + epsilon))

# Map CID to 96 dim features:
CID2features =  {CID: features[i] for i, CID in enumerate(features_CIDs)}

- each `x` in `X` contains a two vector tuple `(mixture_1, mixture_2)`, index ordered same way as `training_set`
- `method` specifies the ways to create the mixture embeeding from 

In [6]:
X, y, num_mixtures, all_pairs_CIDs = format_Xy(training_set,  mixtures_IDs, CID2features, method = 'avg')

In [7]:
# Convert the input pairs to a suitable format for training
X_pairs_1 = np.array([(np.concatenate((x1, x2))) for x1, x2 in X])
X_pairs_2 = np.array([(np.concatenate((x1, x2))) for x2, x1 in X])

y_true = np.array(y)

In [8]:
distances = [get_euclidean_distance(m[0], m[1]) for m in X]
similarities = [get_cosine_similarity(m[0], m[1]) for m in X]
angles = [get_cosine_angle(m[0], m[1]) for m in X]

In [9]:
shared_monos = [ len( set(pair[0]).intersection(set(pair[1]))) for pair in all_pairs_CIDs]
diff_monos = [ len( set(pair[0]).difference(set(pair[1]))) for pair in all_pairs_CIDs]

In [10]:
datasets = training_set['Dataset'].to_numpy()
# Returns the uniques in the order of appearance
desired_order = training_set['Dataset'].unique().tolist() 
encoder = OneHotEncoder(categories=[desired_order])
data_arr = encoder.fit_transform(datasets.reshape(-1, 1))
data_arr = data_arr.toarray()

In [11]:
### add all information above
X_features_1 = np.hstack((X_pairs_1, np.array(distances).reshape(500, 1), 
                        np.array(similarities).reshape(500, 1), 
                        np.array(angles).reshape(500, 1), 
                        np.array(shared_monos).reshape(500, 1), 
                        np.array(diff_monos).reshape(500, 1), 
                        np.array(num_mixtures), 
                        data_arr))
X_features_2 = np.hstack((X_pairs_2, np.array(distances).reshape(500, 1), 
                        np.array(similarities).reshape(500, 1), 
                        np.array(angles).reshape(500, 1), 
                        np.array(shared_monos).reshape(500, 1), 
                        np.array(diff_monos).reshape(500, 1), 
                        np.array(num_mixtures), 
                        data_arr))

In [12]:
X_features = np.empty((1000, X_features_1.shape[1]), dtype=X_features_1.dtype)
X_features[0::2] = X_features_1
X_features[1::2] = X_features_2

y_true= np.repeat(y_true, 2)

In [13]:
# Repeat each row of the training_set dataframe
training_set_repeated = training_set.loc[training_set.index.repeat(2)].reset_index(drop=True)

# Create a new column for the paired index
training_set_repeated['paired_index'] = training_set_repeated.index // 2

# Merge the results with the repeated training_set df
training_set_final = training_set_repeated

# Drop unnecessary columns
training_set_final.drop(['paired_index'], axis=1, inplace=True)

In [14]:
training_set_repeated

,Dataset,Mixture 1,Mixture 2,Experimental Values
0,Snitz 1,1,2,0.604167
1,Snitz 1,1,2,0.604167
2,Snitz 1,1,3,0.651042
3,Snitz 1,1,3,0.651042
4,Snitz 1,1,5,0.505208
...,...,...,...,...
995,Bushdid,515,516,0.730769
996,Bushdid,517,518,0.538462
997,Bushdid,517,518,0.538462
998,Bushdid,519,520,0.807692


----
## 2. Training
### 2.1 Example attempt, standard intialized RF and XGBoost

In [21]:
n_folds = 10
# seed = 314159
seed = 2

In [24]:
best_rf = {'n_estimators': 700, 'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': True}
best_xgb = {'subsample': 0.7, 'n_estimators': 600, 'max_depth': 9, 'learning_rate': 0.01, 'colsample_bytree': 0.7}

Pairing, that indicies were selected such that the two that are the same samples always belong to either train or test.

In [25]:
rf_pred_list = []
xgb_pred_list = []
y_true_list = []
test_indices_list = []

# Create indices for the original samples (before duplication)
original_indices = np.arange(X_features.shape[0] // 2)

# Perform k-fold cross-validation on the original indices:
kf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)

for train_index, test_index in kf.split(original_indices):
    # Convert original indices to the coupled indices
    train_index_coupled = np.concatenate([2*train_index, 2*train_index+1])
    test_index_coupled = np.concatenate([2*test_index, 2*test_index+1])
    
    X_train, X_test = X_features[train_index_coupled], X_features[test_index_coupled]
    y_train, y_test = y_true[train_index_coupled], y_true[test_index_coupled]
    
    # Train the Random Forest regressor
    # rf = RandomForestRegressor(n_estimators=100, random_state=seed)
    rf = RandomForestRegressor(**best_rf, random_state=seed)

    rf.fit(X_train, y_train)
    
    # Train the XGBoost regressor
    # xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=seed)
    xgb_model = xgb.XGBRegressor(**best_xgb, random_state=seed)

    xgb_model.fit(X_train, y_train)
    
    # Make predictions 
    rf_pred = rf.predict(X_test)
    xgb_pred = xgb_model.predict(X_test)
    
    rf_pred_list.extend(rf_pred)
    xgb_pred_list.extend(xgb_pred)
    y_true_list.extend(y_test)
    test_indices_list.extend(test_index_coupled)  # Store the coupled test indices

# Store the predictions and actual values
results_df = pd.DataFrame({
    'test_index': test_indices_list,
    'rf_pred': rf_pred_list,
    'xgb_pred': xgb_pred_list,
    'y_true': y_true_list
})


# Create a temporary index column in training_set_final
training_set_final['original_index'] = range(len(training_set_final))

# Merge the results with the training_set df
training_set_final = training_set_final.merge(results_df, left_on='original_index', right_on='test_index')

# Sort by the original index to restore the original order
training_set_final = training_set_final.sort_values('original_index').reset_index(drop=True)

# Drop unnecessary columns
training_set_final.drop(['original_index', 'test_index'], axis=1, inplace=True)

In [26]:
training_set_final.head()

,Dataset,Mixture 1,Mixture 2,Experimental Values,rf_pred_x,xgb_pred_x,y_true_x,rf_pred_y,xgb_pred_y,y_true_y
0,Snitz 1,1,2,0.604167,0.551580,0.556926,0.604167,0.551771,0.552589,0.604167
1,Snitz 1,1,2,0.604167,0.546400,0.550507,0.604167,0.566216,0.553242,0.604167
2,Snitz 1,1,3,0.651042,0.688436,0.673092,0.651042,0.683967,0.675968,0.651042
3,Snitz 1,1,3,0.651042,0.679467,0.683227,0.651042,0.678366,0.675755,0.651042
4,Snitz 1,1,5,0.505208,0.485055,0.469466,0.505208,0.492298,0.485267,0.505208


In [27]:
# Calculate the correlation and R^2 for Random Forest
rf_corr = np.corrcoef(rf_pred_list, y_true_list)[0, 1]
rf_rmse = np.sqrt(mean_squared_error(np.array(y_true_list), np.array(rf_pred_list)))

print(f"Random Forest - R: {rf_corr:.3f}")
print(f"Random Forest - RMSE: {rf_rmse:.3f}")
print()
# Calculate the correlation and R^2 for XGBoost
xgb_corr = np.corrcoef(xgb_pred_list, y_true_list)[0, 1]
xgb_rmse = np.sqrt(mean_squared_error(np.array(y_true_list), np.array(xgb_pred_list)))

print(f"XGBoost - R: {xgb_corr:.4f}")
print(f"XGBoost - RMSE: {xgb_rmse:.4f}")


Random Forest - R: 0.631
Random Forest - RMSE: 0.123

XGBoost - R: 0.6167
XGBoost - RMSE: 0.1232


In [28]:
# Convert lists to numpy arrays
rf_pred_array = np.array(rf_pred_list)
xgb_pred_array = np.array(xgb_pred_list)
y_true_array = np.array(y_true_list)
test_indices_array = np.array(test_indices_list)

# Create a sorting index based on test_indices_array
sort_idx = np.argsort(test_indices_array)

# Sort all arrays based on this index
rf_pred_sorted = rf_pred_array[sort_idx]
xgb_pred_sorted = xgb_pred_array[sort_idx]
y_true_sorted = y_true_array[sort_idx]
test_indices_sorted = test_indices_array[sort_idx]

# Now, let's pair the sorted arrays
rf_pred_paired = rf_pred_sorted.reshape(-1, 2)
xgb_pred_paired = xgb_pred_sorted.reshape(-1, 2)
y_true_paired = y_true_sorted.reshape(-1, 2)

# Average the pairs
rf_pred_avg = rf_pred_paired.mean(axis=1)
xgb_pred_avg = xgb_pred_paired.mean(axis=1)
y_true_avg = y_true_paired.mean(axis=1)

# Calculate correlations
rf_corr = np.corrcoef(rf_pred_avg, y_true_avg)[0, 1]
xgb_corr = np.corrcoef(xgb_pred_avg, y_true_avg)[0, 1]

# Calculate RMSE
rf_rmse = np.sqrt(mean_squared_error(y_true_avg, rf_pred_avg))
xgb_rmse = np.sqrt(mean_squared_error(y_true_avg, xgb_pred_avg))

# Print results
print("Random Forest Results:")
print(f"Correlation: {rf_corr:.4f}")
print(f"RMSE: {rf_rmse:.4f}")
print("\nXGBoost Results:")
print(f"Correlation: {xgb_corr:.4f}")
print(f"RMSE: {xgb_rmse:.4f}")

# If you need the original indices for these averaged results:
original_indices_avg = test_indices_sorted.reshape(-1, 2).mean(axis=1).astype(int)

Random Forest Results:
Correlation: 0.6312
RMSE: 0.1227

XGBoost Results:
Correlation: 0.6177
RMSE: 0.1231


Result's pretty the same range; which is more reassuring

In [18]:
seeds = list(range(3))
for seed in seeds: 
    print(f"Random search for best hyperparams: round {seed +1} \n")
    rf_best,rbg_best = para_search(seed, X_features, y_true)
    print()
    rf_out = avg_rf_best(rf_best, X_features, y_true)
    print()
    rbg_out = avg_xgb_best(rbg_best, X_features, y_true)
    print()

Random search for best hyperparams: round 1 

Best Random Forest model:
Hyperparameters: {'n_estimators': 200, 'min_samples_split': 5, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': True}
Correlation: 0.9749787928535641
RMSE: 0.04813328596936635

Best XGBoost model:
Hyperparameters: {'subsample': 0.7, 'n_estimators': 700, 'max_depth': 9, 'learning_rate': 0.01, 'colsample_bytree': 0.5}
Correlation: 0.9989799996147316
RMSE: 0.008984191964983554

RandomForest Average Performance (Non-averaged):
R mean: 0.641291744882012
R std: 0.01014620867315997
RMSE mean: 0.12139377372059498
RMSE std: 0.0008219215470271618

RandomForest Average Performance (Averaged):
R mean: 0.6430024603504805
R std: 0.010132916759218572
RMSE mean: 0.12125323140738786
RMSE std: 0.0008204414232010413

RandomForest Overall Performance (Non-averaged):
R: 0.6374800825651145
RMSE: 0.12198836796386285

RandomForest Overall Performance (Averaged):
R: 0.6391073263845439
RMSE: 0.12185005801189092

XGBoost Average Perfo